In [1]:
import cv2
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
from astropy.io import fits
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import probabilistic_hough_line
from skimage.draw import line
from skimage import data
from timeit import default_timer as timer
from scipy.ndimage.filters import gaussian_filter
import numpy.ma as ma
import glob
import os
import pandas as pd
from scipy import ndimage
from skimage.morphology import square, dilation 
import math
import timeit

In [2]:
#Directory where smaps of splitted images and full image is stored
dataset = 'rand_10'
image_folder_smaps = '/lustre/fs23/group/icecube/azenin/last/images/last_streaks/last_' + dataset + '_part/'
full_image_path = '/lustre/fs23/group/icecube/azenin/last/images/last_streaks/last_' + dataset + '_new.fit'
true_mask_path = '/lustre/fs23/group/icecube/azenin/last/images/last_streaks/last_' + dataset + '_mask.fit'
full_image_file = fits.open(full_image_path)
true_mask_file = fits.open(true_mask_path)
full_image = full_image_file[0].data
true_mask = true_mask_file[0].data

#create empty mask for full image
mask = np.zeros((full_image.shape[0],full_image.shape[1]))
xmax, ymax = mask.shape
#properties of subimages and internal stuff
x_parts = 10
y_parts = 5
smap_file = fits.open(image_folder_smaps + 'last_' + dataset + '_x0y0_smap.fit')
smap = smap_file[0].data
cutoff_smaps = np.zeros((x_parts, y_parts, smap.shape[0], smap.shape[1]))
lines_sample = []
global_lines = []

In [3]:
#IMPORTANT: WE NEED PSF (in pixel size), 
psf=1.5

In [4]:
#Load smaps from splitted input image, calculate cutoff
line_array = np.zeros((x_parts, y_parts, 2))
for i in range(x_parts):
    for j in range(y_parts):
        filename = image_folder_smaps + 'last_' + dataset + '_x%d' % i + 'y%d' % j + '_smap.fit' 
        with fits.open(filename) as hdul:
            image=hdul[0].data
            image[np.isnan(image)] = 0
            histogram = image.flatten()
            """
            plt.hist(histogram, 100)
            plt.xlim(0,10)
            plt.yscale('log')
            """
            try:
                histogram=histogram[histogram < 0]
                quantile = np.quantile(histogram, 0.32)
            except:
                quantile = 0
            cutoff = abs(quantile)
            image[image < cutoff] = 0
            cutoff_smaps[i][j] = image
            """
            plt.imshow(image, cmap='gray')
            plt.colorbar()
            plt.show()
            """

In [5]:
global_lines = []
#Do Hough
  
    
for i in range(cutoff_smaps.shape[0]):
    for j in range(cutoff_smaps.shape[1]):
        start2 = timer()
        #edges = canny(cutoff_smaps[i][j], 5*psf, 0.8)
        lines = list(probabilistic_hough_line(cutoff_smaps[i][j], threshold=500, line_length=int(100*psf), line_gap=0))
        for k in range(len(lines)):
            lines[k] = list(lines[k])
            lines[k][0] = list(lines[k][0])
            lines[k][1] = list(lines[k][1])
            lines[k][0][0] += j*smap.shape[1]
            lines[k][1][0] += j*smap.shape[1]
            lines[k][0][1] += i*smap.shape[0]
            lines[k][1][1] += i*smap.shape[0]
#             print("modified: %d, %d; i = %d, j = %d" % (lines[k][0][0], lines[k][0][1], i, j))
#             if(lines[k][0][0] < (i+1) * smap.shape[0] and lines[k][0][1] < (j+1) * smap.shape[1] and lines[k][1][0] < (i+1) * smap.shape[0] and lines[k][1][1] < (j+1) * smap.shape[1]):
            global_lines.append(lines[k])
print(len(global_lines))
#output is list of arrays, where start and end-points of found lines are listed
#not a list of tuples or lists?

1131


In [32]:
###### sort streaks by length
def getdist(line):
    xd = line[0][0] - line[1][0]
    yd = line[0][1] - line[1][1]
    return np.sqrt(np.power(xd, 2) + np.power(yd, 2))

def dfToXY(df_streak):
    x0 = df_streak["x0"]
    x1 = df_streak["x1"]
    y0 = df_streak["y0"]
    y1 = df_streak["y1"]
    return x0, y0, x1, y1

#get slope and intersect for dataframe
def getAB(streak, transpose = 0): 
    x0 = streak[0][1]
    y0 = streak[0][0]
    x1 = streak[1][1]
    y1 = streak[1][0]
    if(transpose):
        x0 = streak[0][0]
        y0 = streak[0][1]
        x1 = streak[1][0]
        y1 = streak[1][1]
    if x0 == x1: # if the line is vertical
        a = np.nan
        b = np.nan
        return a, b, x0, transpose
    else:
        a = (y1 - y0)/(x1 - x0)
        b = y0 - x0 * a
    return a, b, np.nan, transpose

def getAB_t(streak): 
    x0 = streak[0][1]
    y0 = streak[0][0]
    x1 = streak[1][1]
    y1 = streak[1][0]
    transpose = 0
    a = np.nan
    b = np.nan
    if x0 == x1: # if the line is vertical
        transpose = 1
    elif(abs((y1 - y0)/(x1 - x0)) > 1):
        transpose = 1
    if(transpose):
        x0 = streak[0][0]
        y0 = streak[0][1]
        x1 = streak[1][0]
        y1 = streak[1][1]
    a = (y1 - y0)/(x1 - x0)
    b = y0 - x0 * a
    return a, b, np.nan, transpose

# create a pandas dataframe
streaks = np.zeros((len(global_lines), 5))
for i in range(streaks.shape[0]):
    streaks[i][0] = global_lines[i][0][0]
    streaks[i][1] = global_lines[i][0][1]
    streaks[i][2] = global_lines[i][1][0]
    streaks[i][3] = global_lines[i][1][1]
    streaks[i][4] = getdist(global_lines[i])
df = pd.DataFrame(streaks, columns = ["x0", "y0", "x1", "y1", "length"])
df = df.assign(a = np.nan)
df = df.assign(b = np.nan)
df = df.assign(x_vert = np.nan)
df = df.assign(tp = np.nan)
df = df.assign(group = -1)
df = df.assign(group_len = 0)
df = df.assign(streak_len = 0)
df["x0"] = df["x0"].astype(np.int32)
df["y0"] = df["y0"].astype(np.int32)
df["x1"] = df["x1"].astype(np.int32)
df["y1"] = df["y1"].astype(np.int32)
df = df.sort_values("length", ascending = [False]).reset_index(drop=True)
for i in range(len(df)):
    x0, y0, x1, y1 = dfToXY(df.loc[i])
    a, b, xx, tp = getAB_t([[x0, y0], [x1, y1]])
#     a, b, xx, tp = getAB([[x0, y0], [x1, y1]])
    df.at[i, "a"] = a
    df.at[i, "b"] = b
    df.at[i, "x_vert"] = xx
    df.at[i, "tp"] = tp
df
# df.dtypes

,x0,y0,x1,y1,length,a,b,x_vert,tp,group,group_len,streak_len
0,3809,8810,2540,8699,1273.845360,0.087470,8476.825059,NaN,1.0,-1,0,0
1,5079,8918,3810,8807,1273.845360,0.087470,8473.737589,NaN,1.0,-1,0,0
2,3809,8811,2540,8700,1273.845360,0.087470,8477.825059,NaN,1.0,-1,0,0
3,5182,3827,5957,2871,1230.675018,-0.810669,8284.432008,NaN,0.0,-1,0,0
4,5181,3827,5955,2871,1230.045528,-0.809623,8279.428870,NaN,0.0,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1126,4703,8875,4552,8865,151.330764,0.066225,8563.543046,NaN,1.0,-1,0,0
1127,4509,8858,4358,8848,151.330764,0.066225,8559.390728,NaN,1.0,-1,0,0
1128,6042,6886,6026,6736,150.850920,0.106667,5307.493333,NaN,0.0,-1,0,0
1129,798,8557,648,8544,150.562279,0.086667,8487.840000,NaN,1.0,-1,0,0


In [7]:
# plot the result
%matplotlib widget
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(full_image, cmap=cm.gray, norm=LogNorm())
ax[0].set_title('Input image')

ax[1].imshow(full_image * 0)
for i in range(df.shape[0]):
    p0, p1 = [df["x0"][i], df["y0"][i]], [df["x1"][i], df["y1"][i]]
    ax[1].plot((float(p0[0]), float(p1[0])), (float(p0[1]), float(p1[1])))
    #ax[1].set_xlim((0,full_image.shape[1]))
#ax[1].set_ylim((full_image.shape[0], 0))
ax[1].set_title('Probabilistic Hough')

# for a in ax:
#     a.set_axis_off()

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# define some useful functions

def drawLine(image, x0, y0, x1, y1):
    xl, yl = line(x0, y0, x1, y1)
    image[xl, yl] = 1

# obtains parameters of line equation from two points
# def getab(streak, transpose = 0): 
#     x0 = streak[0][1]
#     y0 = streak[0][0]
#     x1 = streak[1][1]
#     y1 = streak[1][0]
#     if(transpose):
#         x0 = streak[0][0]
#         y0 = streak[0][1]
#         x1 = streak[1][0]
#         y1 = streak[1][1]
#     if x0 == x1: # if the line is vertical
#         a = np.nan
#         b = np.nan
#         return a, b, x0
#     else:
#         a = (y1 - y0)/(x1 - x0)
#         b = y0 - x0 * a
#     return a, b, -1
    
# extends a line to the edges of the image (still needs testing for larger angles)
# def extendMask(image, x0, y0, x1, y1):
#     a, b, xx = getab([[x0, y0], [x1, y1]], transpose = 0)
#     if abs(a) <= 1:
#         transpose = 0
#         xmax = full_image.shape[0]
#         ymax = full_image.shape[1]
#     else:
#         a, b, xx = getab([[x0, y0], [x1, y1]], transpose = 1)
#         transpose = 1
#         xmax = full_image.shape[1]
#         ymax = full_image.shape[0]
#         image = np.transpose(image)
#         print('transposed')
    
#     print("a = %f, b = % f, xx = %f" % (a,b,xx))
#     if xx == -1.:
#         print('line is not vertical')
#         if np.absolute(a) <= 1.:
#             print('a <= 1')
#             for i in range(0, xmax):
#                 if(int(a * i + b) > 0 and int(a * i + b) < ymax):
#                     image[i][int(a * i + b)] = 1.
                        
#     else:
#         for i in range(im_w):
#             image[i][xx] = 1
            
#     if transpose:
#         image = np.transpose(image)

# plt.imshow(base_mask)
# print(np.count_nonzero(base_mask))

In [43]:
def combine_streak(str_num = 0):
    newmask = np.zeros((full_image.shape[0],full_image.shape[1]), dtype=np.bool_)
    if(len(df.index[df['group'] == str_num].tolist())):
        print("streak number %d already exists" % str_num)
        return newmask, 0
    indices = df.index[df['group'] == -1].tolist()
    if(len(indices) == 0):
        print("no more lines to check")
        return newmask, -1
    print('\nlongest line: i = %d, len = %f' % (indices[0], df.at[indices[0], 'length']))

    chng_cntr = 0
    group_len = 0
    #calculate alignment
    slope_tol_deg = 2
    slope_tol = slope_tol_deg * np.pi / 180
    intersect_tol = 200
    print('%d streaks to check' % len(indices))
    tochange = df[df['group'] == -1].index[np.isclose(np.arctan(df["a"].loc[indices]), 
                                   np.arctan(df["a"].loc[indices[0]]), 
                                   atol = slope_tol) 
                        & np.isclose(df["b"].loc[indices],
                                     df["b"].loc[indices[0]],
                                     atol = intersect_tol)
                        & np.asarray(df["tp"].loc[indices] == 0)]
    if len(tochange) == 0:
        tochange = df[df['group'] == -1].index[np.isclose(np.arctan(df["a"].loc[indices]), 
                                       np.arctan(df["a"].loc[indices[0]]), 
                                       atol = slope_tol) 
                            & np.isclose(df["b"].loc[indices],
                                         df["b"].loc[indices[0]],
                                         atol = intersect_tol)
                            & np.asarray(df["tp"].loc[indices] == 1)]

    df.at[tochange, "group"] = str_num
    df.at[tochange, "group_len"] = np.sum(df.loc[tochange]["length"])
    chng_cntr = len(tochange)
    print('got %d aligned streaks' % chng_cntr)
    
    
    #if no correlations, exit
    if(chng_cntr == 0):
        df.at[indices[0], "group"]= -10
        return newmask, chng_cntr
    if(chng_cntr <= 10):
        print('too few streaks')
        indices = df.index[df['group'] == str_num].tolist()
        for i in range(len(indices)):
            df.at[indices[i], "group"] = -11
            df.at[indices[i], "group"] = -11
        return newmask, 0
    
    #make a new line
    sdf = df[df['group'] == str_num]
    min_y = np.min([sdf['y0'].min(), sdf['y1'].min()])
    max_y = np.max([sdf['y0'].max(), sdf['y1'].max()])
    if sdf['y0'].min() < sdf['y1'].min():
        min_x = np.mean(sdf[sdf["y0"] == min_y]['x0'])
        max_x = np.mean(sdf[sdf["y1"] == max_y]['x1'])
    else:
        min_x = np.mean(sdf[sdf["y1"] == min_y]['x1'])
        max_x = np.mean(sdf[sdf["y0"] == max_y]['x0'])
    drawLine(newmask, int(min_y), int(min_x), int(max_y), int(max_x))
    xd = max_x - min_x
    yd = max_y - min_y
    streak_len = np.sqrt(np.power(xd, 2) + np.power(yd, 2))
     
    df.at[tochange, "streak_len"] = streak_len
#     actual_len = np.count_nonzero(newmask)
#     if newm_corr < altm_corr:
#         newmask = newmask_alt
#         newm_corr = altm_corr
#     if newm_corr / np.count_nonzero(newmask) < 0.9:
#         print('does not correlate well')
#         indices = df.index[df['group'] == str_num].tolist()
#         for i in range(len(indices)):
#             df.at[indices[i], "group"] = -12
#         newmask = np.zeros((full_image.shape[0],full_image.shape[1]), dtype=np.bool_)
#         return newmask, 0

#     indices = df.index[df['group'] == str_num].tolist()
#     for i in range(len(indices)):
#         df.at[indices[i], "group_len"] = group_len
#         df.at[indices[i], "streak_len"] = actual_len
#     print('group %d: integrated length = %f; actual length = %f' % (str_num, group_len, actual_len))
#     if(group_len < actual_len):
#         print('huge length difference')
#         indices = df.index[df['group'] == str_num].tolist()
#         for i in range(len(indices)):
#             df.at[indices[i], "group"] = -12
#         newmask = np.zeros((full_image.shape[0],full_image.shape[1]), dtype=np.bool_)
#         return newmask, 0
    
    return newmask, chng_cntr

In [41]:
df["group"] = -1
indices = df.index[df['group'] == -1].tolist()
np.asarray(df["tp"].loc[indices] == 1)
# np.isclose(df["b"].loc[indices],
#                                      df["b"].loc[indices[0]],
#                                      atol = 300)

array([ True,  True, False, ...,  True, False,  True])

In [63]:
def plotStreakGroup(canvas, str_num):
    indices = df.index[df['group'] == str_num].tolist()
    if len(indices) == 0:
        print('the group is empty, nothing to draw')
        return
    print('drawing group %d' % str_num)
    for i in indices:
        drawLine(canvas, 
                df["y0"][i],
                df["x0"][i],
                df["y1"][i],
                df["x1"][i])
    
    
st_num = 4
group_mask = np.zeros((full_image.shape[0], full_image.shape[1]), dtype = np.bool_)
plotStreakGroup(group_mask, st_num)
struct2 = ndimage.generate_binary_structure(2, 2)
group_mask = ndimage.binary_dilation(group_mask, structure=struct2, iterations = 30).astype(group_mask.dtype)
%matplotlib widget
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(base_mask)
    #ax[1].set_xlim((0,full_image.shape[1]))
#ax[1].set_ylim((full_image.shape[0], 0))
ax[0].set_title('Base mask')
ax[1].imshow(group_mask)
ax[1].set_title('group number %d' % st_num)
ax[2].imshow(true_mask)
ax[2].set_title('True mask')
df[df['group'] == st_num]

drawing group 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,x0,y0,x1,y1,length,a,b,x_vert,tp,group,group_len,streak_len
56,1271,8612,1801,7656,1093.085541,-0.554393,6045.435146,NaN,0.0,4,64060.371532,5718.086328
57,1273,8612,1803,7656,1093.085541,-0.554393,6047.435146,NaN,0.0,4,64060.371532,5718.086328
58,1274,8612,1804,7656,1093.085541,-0.554393,6048.435146,NaN,0.0,4,64060.371532,5718.086328
59,1275,8612,1804,7658,1090.851502,-0.554507,6050.417191,NaN,0.0,4,64060.371532,5718.086328
60,2831,5741,3339,4785,1082.589488,-0.531381,5881.656904,NaN,0.0,4,64060.371532,5718.086328
...,...,...,...,...,...,...,...,...,...,...,...,...
1030,2540,6281,2621,6128,173.118457,-0.529412,5865.235294,NaN,0.0,4,64060.371532,5718.086328
1033,2639,6096,2720,5944,172.235304,-0.532895,5887.526316,NaN,0.0,4,64060.371532,5718.086328
1038,1053,9019,1134,8868,171.353436,-0.536424,5891.006623,NaN,0.0,4,64060.371532,5718.086328
1046,3555,4387,3635,4236,170.883001,-0.529801,5879.238411,NaN,0.0,4,64060.371532,5718.086328


In [44]:
# %%timeit
def calc_groups():
    df["group"] = -1 # to reset
    base_mask = np.zeros((full_image.shape[0], full_image.shape[1]), dtype = np.bool_)
    streak_counter = 0
    fail_counter = 0
    while(True):
        newmask, chng_counter = combine_streak(streak_counter)
    #     if chng_counter == 0:
    #         fail_counter += 1
    #         if fail_counter > 10:
    #             break
    #     else:
        if chng_counter == -1:
            break
        elif chng_counter == 0:
            continue
        fail_counter = 0
        streak_counter += 1
        base_mask += newmask
    print(streak_counter)
    struct2 = ndimage.generate_binary_structure(2, 2)
    base_mask = ndimage.binary_dilation(base_mask, structure=struct2, iterations = 5).astype(base_mask.dtype)
    return streak_counter, base_mask
        # plt.imshow(newmask)
        # plt.show()
streak_counter, base_mask = calc_groups()


longest line: i = 0, len = 1273.845360
1131 streaks to check
got 175 aligned streaks

longest line: i = 3, len = 1230.675018
956 streaks to check
got 119 aligned streaks

longest line: i = 17, len = 1153.239351
837 streaks to check
got 96 aligned streaks

longest line: i = 48, len = 1127.625824
741 streaks to check
got 110 aligned streaks

longest line: i = 56, len = 1093.085541
631 streaks to check
got 151 aligned streaks

longest line: i = 62, len = 1063.966635
480 streaks to check
got 160 aligned streaks

longest line: i = 86, len = 1017.369156
320 streaks to check
got 63 aligned streaks

longest line: i = 90, len = 1008.885524
257 streaks to check
got 68 aligned streaks

longest line: i = 91, len = 999.848989
189 streaks to check
got 40 aligned streaks

longest line: i = 149, len = 833.255063
149 streaks to check
got 34 aligned streaks

longest line: i = 236, len = 610.462939
115 streaks to check
got 4 aligned streaks
too few streaks

longest line: i = 273, len = 555.360243
111 st

In [45]:
# df.loc[47]
%matplotlib widget
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True)
ax = axes.ravel()
print(streak_counter)
ax[0].imshow(full_image * 0)
for i in range(df.shape[0]):
    p0, p1 = [df["x0"][i], df["y0"][i]], [df["x1"][i], df["y1"][i]]
    ax[0].plot((float(p0[0]), float(p1[0])), (float(p0[1]), float(p1[1])))
    #ax[1].set_xlim((0,full_image.shape[1]))
#ax[1].set_ylim((full_image.shape[0], 0))
ax[0].set_title('Probabilistic Hough')
ax[1].imshow(base_mask)
ax[1].set_title('Calculated mask')
ax[2].imshow(true_mask)
ax[2].set_title('True mask')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

12


In [ ]:
canvas = np.zeros((full_image.shape[0],full_image.shape[1])).astype(np.bool_)
index = 640
drawLine(canvas, 
         df["y0"][index],
         df["x0"][index], 
         df["y1"][index],
         df["x1"][index])
struct2 = ndimage.generate_binary_structure(2, 2)
canvas = ndimage.binary_dilation(canvas, structure=struct2, iterations = 50).astype(canvas.dtype)
%matplotlib widget
plt.imshow(canvas)
plt.show()

In [ ]:
# %matplotlib widget
# base_mask = np.zeros((full_image.shape[0],full_image.shape[1])).astype(np.bool_)
# extendMask(base_mask, 
#            df["x0"][4], 
#            df["y0"][4], 
#            df["x1"][4], 
#            df["y1"][4])
# struct2 = ndimage.generate_binary_structure(2, 2)
# base_mask = ndimage.binary_dilation(base_mask, structure=struct2, iterations = 100).astype(base_mask.dtype)
# plt.imshow(base_mask)
# plt.show()
%matplotlib widget
im_w = full_image.shape[0]
im_h = full_image.shape[1]
mask = np.zeros((im_w, im_h), dtype = np.bool_)
indices = df.index[df['group'] == -1].tolist()
drawLine(mask, 
         df["y0"][indices[0]],
         df["x0"][indices[0]], 
         df["y1"][indices[0]],
         df["x1"][indices[0]])
print('longest line: i = %d, len = %f' % (indices[0], df.at[indices[0], 'length']))

%matplotlib widget
base_mask = np.zeros((im_w, im_h)).astype(np.bool_)
extendMask(base_mask, 
           df["x0"][indices[0]], 
           df["y0"][indices[0]], 
           df["x1"][indices[0]], 
           df["y1"][indices[0]])
# make the line wider and find where it's not zero
struct2 = ndimage.generate_binary_structure(2, 2)
base_mask = ndimage.binary_dilation(base_mask, structure=struct2, iterations = 100).astype(base_mask.dtype)
plt.imshow(base_mask)
plt.show()

In [ ]:
%matplotlib widget
newline = np.zeros((full_image.shape[0],full_image.shape[1]), dtype = np.bool_)
drawLine(newline,
         df["y0"][3], 
         df["x0"][3],
         df["y1"][3], 
         df["x1"][3])
struct2 = ndimage.generate_binary_structure(2, 2)
newline = ndimage.binary_dilation(newline, structure=struct2, iterations = 20).astype(newline.dtype)
plt.imshow(newline)
plt.show()
print(df.at[2, 'length'])

In [ ]:
%matplotlib widget
plt.imshow(cutoff_smaps[1][1])
plt.colorbar()
plt.show()

In [ ]:
full_smap_file = fits.open("/lustre/fs23/group/icecube/azenin/last/images/snr2_smap.fit")
full_smap = full_smap_file[0].data
image = full_smap
histo = image.flatten()
try:
    histo=histo[histogram < 0]
    quantile = np.quantile(histo, 0.32)
except:
    quantile = 0
cutoff = abs(quantile)
image[image < cutoff] = 0
plt.imshow(image)
plt.colorbar()
plt.show()


In [46]:
### efficiency metric
# make an expectation array with known streak parameters
# streak in images (x = ay + b):
# true_expanded = ndimage.binary_dilation(true_mask, structure=struct2, iterations = 1).astype(np.int16)
true_expanded = true_mask
inv_true_mask = np.ones((full_image.shape[0], full_image.shape[1]))
inv_mask = np.ones((full_image.shape[0], full_image.shape[1]))

inv_true_mask -= true_expanded
inv_mask -= base_mask
# compare with the result
targetpos = np.count_nonzero(true_expanded > 0.1)
targetneg = np.count_nonzero(inv_true_mask > 0.1)
truepos = np.sum(base_mask[true_expanded > 0.1])
falsepos = np.sum(base_mask[true_expanded < 0.1])
efficiency_tp = truepos / targetpos
efficiency_fp = falsepos / targetneg 
print("true positives: %f %%" % (efficiency_tp*100))
print("false positives: %f %%" % (efficiency_fp*100))

true positives: 79.533037 %
false positives: 0.234004 %


In [ ]:
%matplotlib widget
plt.imshow(expectation_mask)
plt.colorbar()
plt.show()

In [ ]:
# plt.imshow(mask)
%matplotlib widget
# plt.imshow(mask[0].astype(float))
plt.plot(mask[0][1000, :])
plt.show()

In [ ]:
df["x0"][2]

In [ ]:
print(im_h)

In [ ]:
df[df['group'] == -1]

In [ ]:
indices = df.index[df['group'] == -1].tolist()
indices

In [ ]:
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
h, theta, d = hough_line(full_image, theta=tested_angles)

In [ ]:
makeRefCone(df[df['group']==1], psf)

In [ ]:
df[df['group']==-1]

In [ ]:
ref = makeRefCone(df.loc[0], psf)

In [ ]:
ref

In [ ]:
df.loc[0]

In [ ]:
df[np.isclose(np.arctan(df["a"]), np.arctan(df["a"].iloc[0]), atol = 10) & np.isclose(df["b"], 2600, atol=50)]

In [ ]:
indices = df.index[df['group'] == -1].tolist()

In [ ]:
df.loc[indices]

In [ ]:
df.index[np.isclose(np.arctan(df["a"].loc[indices]), np.arctan(df["a"].loc[indices[0]]), atol = 10.) & np.isclose(df["b"], df["b"].loc[indices[0]], rtol = 0.1)]

In [ ]:
sdf = df[df['group'] == -1]

In [ ]:
min_y = np.min([sdf['y0'].min(), sdf['y1'].min()])
max_y = np.max([sdf['y0'].max(), sdf['y1'].max()])
if sdf['y0'].min() < sdf['y1'].min():
    min_x = np.mean(sdf[sdf["y0"] == min_y]['x0'])
    max_x = np.mean(sdf[sdf["y1"] == max_y]['x1'])
else:
    min_x = np.mean(sdf[sdf["y1"] == min_y]['x1'])
    max_x = np.mean(sdf[sdf["y0"] == max_y]['x0'])

In [ ]:
tochange = df.index[np.isclose(np.arctan(df["a"].loc[indices]), 
                                   np.arctan(df["a"].loc[indices[0]]), 
                                   atol = 10.) 
                        & np.isclose(df["b"].loc[indices],
                                     df["b"].loc[indices[0]],
                                     rtol = 0.1)]

In [ ]:
df[df["group"] == 0]

In [ ]:
indices = df.index[df['group'] == -1].tolist()
tochange = df[df['group'] == -1].index[np.isclose(np.arctan(df["a"].loc[indices]), 
                                   np.arctan(df["a"].loc[indices[0]]), 
                                   atol = 10.) 
                        & np.isclose(df["b"].loc[indices],
                                     df["b"].loc[indices[0]],
                                     rtol = 0.1)]

In [ ]:
df["a"].loc[indices[0]]

In [ ]:
len(np.isclose(np.arctan(df["a"].loc[indices]), 
                                   np.arctan(df["a"].loc[indices[0]]), 
                                   atol = 10.)) 

In [53]:
df[df['group'] == 10]

,x0,y0,x1,y1,length,a,b,x_vert,tp,group,group_len,streak_len
544,2661,6046,2808,5758,323.346563,-0.510417,5746.979167,NaN,0.0,10,2530.594163,3965.059016
624,2677,6017,2808,5761,287.570861,-0.511719,5756.011719,NaN,0.0,10,2530.594163,3965.059016
661,2687,6000,2808,5762,266.992509,-0.508403,5737.420168,NaN,0.0,10,2530.594163,3965.059016
673,1147,8851,1269,8621,260.353606,-0.530435,5841.878261,NaN,0.0,10,2530.594163,3965.059016
674,1147,8852,1269,8622,260.353606,-0.530435,5842.408696,NaN,0.0,10,2530.594163,3965.059016
776,2542,6285,2647,6079,231.216349,-0.509709,5745.519417,NaN,0.0,10,2530.594163,3965.059016
932,941,9256,1027,9088,188.732615,-0.511905,5679.190476,NaN,0.0,10,2530.594163,3965.059016
933,975,9192,1061,9024,188.732615,-0.511905,5680.428571,NaN,0.0,10,2530.594163,3965.059016
981,1183,8780,1267,8621,179.824915,-0.528302,5821.490566,NaN,0.0,10,2530.594163,3965.059016
1023,2558,6256,2637,6101,173.971262,-0.509677,5746.541935,NaN,0.0,10,2530.594163,3965.059016
